Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

## Deploy Tiny YOLOv2 Real-time Object Detection using ONNX on AzureML

This notebook guides you to register ONNX TinyYOLO model in Azure Machine Learning Service, deploy it as a web service, and run inference by providing input images.

### Download Tiny YOLOv2 Models

Tiny YOLO is a real-time neural network for object detection. We download the model file from ONNX Zoo. More details about ONNX Tiny YOLO model can be found at: https://github.com/onnx/models/tree/master/tiny_yolov2

In [ ]:
import time

start = time.time()
!wget -nc -P model/  https://onnxzoo.blob.core.windows.net/models/opset_8/tiny_yolov2/tiny_yolov2.tar.gz
!tar xvzf model/tiny_yolov2.tar.gz -C model/
end = time.time()
print("Model download latency: {} seconds".format(end-start))

### Load AML Workspace

We will use the Azure Machine Learning Service Workspace created by running configuration.ipynb. Please run that notebook first if you haven't yet.

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()
print(ws.name, ws.subscription_id, ws.resource_group, ws.location)

### Register Tiny YOLO Model in Azure Machine Learning Service

Run following python code snippets to register tiny YOLOv2 model to Model Management Service provided by Azure Machine Learning Service.

In [ ]:
from azureml.core.model import Model

start = time.time()
models = []

new_model = Model.register(
    workspace = ws,
    model_path = "model/tiny_yolov2/model.onnx",
    model_name = "tinyyolov2",
    tags = {"onnx": "demo"})
models.append(new_model)

end = time.time()
print("Register model latency: {} seconds".format(end-start))
print("Model id={}, name={}, created time={}".format(
    new_model.id, 
    new_model.name, 
    new_model.created_time))

### Build Container Image 

Next, we will create a docker image which runs a flask web service. The web service has multiple endpoints to serve the tinyYOLO-v2 models.

Please **Note**: this web service contains javascript codes to "reshape" the input image to 416 X 416 pixels in order to run Tiny YOLO model, and the logic to "reshape" the output classfication rectangle boxes back to the original image size. Details are under js folder.

In [ ]:
from azureml.core.image import Image, ContainerImage

image_config = ContainerImage.image_configuration(
    execution_script = "score.py",
    runtime = "python", 
    dependencies = ["app.py", "ui.html", "static/bundle.js"],
    conda_file = "env.yml",
    tags = {"onnx": "demo"})

start = time.time()
image = Image.create(
    workspace = ws,
    name = "onnx-yolo-demo-image",
    models = models,
    image_config = image_config
)

image.wait_for_creation(show_output = True)

end = time.time()
print("Image creation latency: {} seconds".format(end-start))

### Deploy Image to ACI

After we successfully create the image, we will use Azure Machine Leaning's python SDK to deploy it via Azure Container Instances (ACI)

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

aci_config = AciWebservice.deploy_configuration(cpu_cores = 1, memory_gb = 1)

start = time.time()
aci_service = Webservice.deploy_from_image(
    workspace = ws,
    name = "onnx-yolo-demo-service2",
    image = image,
    deployment_config = aci_config)
aci_service.wait_for_deployment(show_output = True)

end = time.time()
print("Service deployment latency: {} seconds".format(end-start))

if (aci_service.state != 'Healthy'):
    raise ValueError("service deployment is in bad state: {}".format(aci_service.state))

### Run Inference Interactively

The deployed web service provides a UI page. You can drag and drop image and receive the predictions in real-time, shown as rectangle boxes overlapped to the image. 

You can use the sample images under SampleImages folder to try out the inference. 

In [ ]:
# Get demo URL 
demo_uri = aci_service.scoring_uri.replace("/score", "/ui")
print("Try out the ONNX TinyYOLO model at {}".format(demo_uri))

### Cleanup Resources

Delete the web serivce, image, and models

In [ ]:
aci_service.delete()
image.delete()
for m in models:
    m.delete()
print("Successfully deleted models, image and service.")